In [1]:
import os
%mkdir "train/dog"
%mkdir "train/cat"

mkdir: cannot create directory ‘train/dog’: File exists
mkdir: cannot create directory ‘train/cat’: File exists


First things is to put files into directories by category so I can use DataImageGenerator from keras.  Pretty easy as the category name is in the file name.

In [2]:
for dir, subdir, files in os.walk("train"):
    if len(subdir) == 0:
        continue
    for file in files:
        category = file.split(".")[0]
        os.rename("{}/{}".format(dir,file), "{}/{}/{}".format(dir, category, file))

Now make a validation set by picking samples at random and placing them in the validiation folder 'valid'.  Note that this selects images at random only once--a sort of permanent split for the project.  

In [ ]:
%mkdir "valid"
%mkdir "valid/dog"
%mkdir "valid/cat"

In [ ]:
dogs = [x for x in os.listdir("train/dog")]
cats = [x for x in os.listdir("train/cat")]


In [ ]:
import random
if len(os.listdir("valid/dog")) < 1:
    for n in random.sample(range(len(dogs)), 1000):
        os.rename("train/dog/{}".format(dogs[n]), "valid/dog/{}".format(dogs[n]))

if len(os.listdir("valid/cat")) < 1:
    for n in random.sample(range(len(cats)), 1000):
        os.rename("train/cat/{}".format(cats[n]), "valid/cat/{}".format(cats[n]))

Here is the meat of the program.  First I create a generator to stream data into the neural net
the train_generator 'generator' can also include commands to flip or rotate images at random.

In [1]:
ROWS = 128
COLS = 128
CHANNELS = 3
BATCH_SIZE=32

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',
        rotation_range=30.)

validation_generator = test_datagen.flow_from_directory(
        'valid',
        target_size=(ROWS, COLS),
        batch_size=BATCH_SIZE,
        class_mode='binary')

Using TensorFlow backend.


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Here is the net architecture.
2 Convolution layers, first one has 4x4 stide, second 3x3.  16 filters each.
Standard pooling (2x2)
2 more convolution layers with 32 filters (3x3 stride)
Pooling layer
Dense layer with 50% dropout (overfitting prevention) 
Output

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Dense, Activation, Flatten, Dropout, MaxPooling2D
from keras.regularizers import l2

model = Sequential()
model.add(Convolution2D(16, 4, 4, border_mode='same', input_shape=(ROWS,COLS,CHANNELS),activation='relu'))
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), border_mode='same'))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), border_mode='same'))
model.add(Flatten())
model.add(Dense(output_dim=64, W_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(output_dim=1, W_regularizer=l2(0.01)))  #binary classification
model.add(Activation('relu'))

Run that net! Here I am only using 2000 samples (mainly for speed).  Gives acceptable results in 5 minutes.  Going to all samples provides increased accuracy, but of course, takes much longer.

In [4]:
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
model.fit_generator(generator=train_generator,samples_per_epoch=2000,nb_epoch=20)

Epoch 1/20
2016/2000 [==============================] - 26s - loss: 0.9549 - acc: 0.5615    

/home/robert/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Epoch 2/20
2016/2000 [==============================] - 25s - loss: 0.9084 - acc: 0.5615    
Epoch 3/20
2016/2000 [==============================] - 25s - loss: 0.8052 - acc: 0.5967    
Epoch 4/20
2016/2000 [==============================] - 25s - loss: 0.7924 - acc: 0.6151    
Epoch 5/20
2016/2000 [==============================] - 25s - loss: 0.7936 - acc: 0.6181    
Epoch 6/20
2016/2000 [==============================] - 26s - loss: 0.7764 - acc: 0.6017    
Epoch 7/20
2016/2000 [==============================] - 26s - loss: 0.7572 - acc: 0.6295    
Epoch 8/20
2016/2000 [==============================] - 25s - loss: 0.7340 - acc: 0.6225    
Epoch 9/20
2016/2000 [==============================] - 25s - loss: 0.7142 - acc: 0.6434    
Epoch 10/20
2008/2000 [==============================] - 24s - loss: 0.6665 - acc: 0.6713    
Epoch 11/20
2016/2000 [==============================] - 12s - loss: 0.6480 - acc: 0.6746    
Epoch 12/20
2016/2000 [==============================] - 12s - loss

In [5]:
print("valid set :", model.evaluate_generator(validation_generator,val_samples=2000)[1]*100, "%")
print("--------------------")
print("train set :", model.evaluate_generator(train_generator,val_samples=2000)[1]*100, "%")


valid set : 67.05 %
--------------------
train set : 68.75 %


In [ ]:
model.evaluate_generator(train_generator,val_samples=2000)